<a href="https://colab.research.google.com/github/GuilhermePelegrina/Mackenzie/blob/main/Aulas/2025_1s/TIC/Extra_Pipeline_ColumnTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Comandos Importantes do Scikit Learn

Até agora vc aprendeu a resolver o problema de classificação da seguinte forma:

### 1. **Importando as Bibliotecas**
Primeiro, importamos as bibliotecas necessárias.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

### 2. **Carregando o Dataset**
Vamos carregar o dataset Titanic e visualizar os dados.

In [ ]:
import seaborn as sns
df = sns.load_dataset('titanic')

In [ ]:
# Filtrando colunas relevantes e removendo valores nulos
df = df[['sex', 'embarked', 'age', 'fare', 'survived']].dropna()

In [ ]:
# Exibir as primeiras linhas do dataset
df.head()

,sex,embarked,age,fare,survived
0,male,S,22.0,7.2500,0
1,female,C,38.0,71.2833,1
2,female,S,26.0,7.9250,1
3,female,S,35.0,53.1000,1
4,male,S,35.0,8.0500,0


### 3. **Dividindo o Dataset em Treinamento e Teste**
Dividimos o conjunto de dados em variáveis independentes (X) e a variável dependente (y), e, em seguida, dividimos em conjuntos de treino e teste.


In [ ]:
# Dividindo em variáveis independentes (X) e dependente (y)
X = df[['sex', 'embarked', 'age', 'fare']]
y = df['survived']


In [ ]:
# Dividindo em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


### 4. **Aplicando OneHotEncoder para Variáveis Categóricas**
Agora, aplicamos a codificação **OneHotEncoder** às variáveis categóricas `sex` e `embarked` manualmente.

In [ ]:
# Definindo as colunas categóricas
categorical_cols = ['sex', 'embarked']

In [ ]:
# Criando o OneHotEncoder
one_hot_encoder = OneHotEncoder(drop='first', sparse_output=False)

In [ ]:
# Ajustando e transformando o treino
X_train_cat_encoded = one_hot_encoder.fit_transform(X_train[categorical_cols])

In [ ]:
# Aplicando as mesmas transformações ao conjunto de teste
X_test_cat_encoded = one_hot_encoder.transform(X_test[categorical_cols])

In [ ]:
# Verificando as transformações
print(X_train_cat_encoded[:5])  # Primeiros 5 exemplos codificados

[[0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 1.]
 [1. 0. 0.]
 [0. 0. 0.]]


### 5. **Normalizando Variáveis Numéricas com StandardScaler**
As variáveis numéricas `age` e `fare` precisam ser normalizadas usando **StandardScaler**.

In [ ]:
# Definindo as colunas numéricas
numeric_cols = ['age', 'fare']

In [ ]:
# Criando o StandardScaler
scaler = StandardScaler()

In [ ]:
# Ajustando e transformando o treino
X_train_num_scaled = scaler.fit_transform(X_train[numeric_cols])

In [ ]:
# Aplicando as mesmas transformações ao conjunto de teste
X_test_num_scaled = scaler.transform(X_test[numeric_cols])

In [ ]:
# Verificando as normalizações
print(X_train_num_scaled[:5])  # Primeiros 5 exemplos escalonados

[[ 0.24100362 -0.09238956]
 [ 0.87559868 -0.12995881]
 [-0.32308088 -0.40904465]
 [ 0.73457755 -0.09301643]
 [-0.74614426  1.26717071]]


### 6. **Concatenando Variáveis Codificadas e Normalizadas**
Agora, combinamos as variáveis categóricas codificadas e as variáveis numéricas normalizadas.

In [ ]:
# Concatenando as variáveis categóricas e numéricas
X_train_transformed = np.hstack([X_train_cat_encoded, X_train_num_scaled])
X_test_transformed = np.hstack([X_test_cat_encoded, X_test_num_scaled])

In [ ]:
# Verificando as dimensões resultantes
print(X_train_transformed.shape)

(569, 5)


In [ ]:
### 7. **Treinando o Modelo de Regressão Logística**
## Usamos os dados transformados para treinar o modelo de Regressão Logística.

In [ ]:
# Criando o modelo de Regressão Logística
model = LogisticRegression()

In [ ]:
# Ajustando o modelo com o conjunto de treino
model.fit(X_train_transformed, y_train)

LogisticRegression()

### 8. **Fazendo Previsões e Avaliando o Modelo**
Finalmente, fazemos as previsões no conjunto de teste e avaliamos a acurácia do modelo.

In [ ]:
# Prevendo os resultados no conjunto de teste
y_pred = model.predict(X_test_transformed)

In [ ]:
# Avaliando a acurácia
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia do modelo: {accuracy:.2f}")

Acurácia do modelo: 0.76


### Resumo:
- **OneHotEncoder** foi aplicado manualmente às colunas categóricas.
- **StandardScaler** foi utilizado para normalizar as colunas numéricas.
- As transformações de treino e teste foram feitas de maneira explícita para garantir que o conjunto de teste tenha a mesma escala e codificação do treino.
- Um modelo de **LogisticRegression** foi treinado e avaliado com uma acurácia final mostrada.


# Agora com Estilo!

Vamos agora utilizar utilizar o comando `Pipeline` do scikit-learn para automatizar o processo de One Hot Encoding e Normalização (Scaler).

O uso de `Pipeline` torna o código mais organizado e garante que todas as etapas de pré-processamento sejam aplicadas corretamente ao conjunto de dados de treinamento e teste.

In [ ]:
# Classificação e Encoding de Variáveis Categóricas com One Hot Encoding
# Aplicação Correta do fit e transform usando Pipeline

# Seção 1: Introdução
"""
Nesta aula, vamos aprender a usar o One Hot Encoding para variáveis categóricas e como usar corretamente
a sequência de .fit() e .transform() em um Pipeline para aplicar as transformações no conjunto de treino e teste.
Além disso, vamos combinar a normalização de variáveis numéricas no mesmo Pipeline.
"""

# Seção 2: Importando Bibliotecas Necessárias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [ ]:
# Seção 3: Carregando o Dataset e Explorando as Variáveis Categóricas
"""
Vamos usar o dataset Titanic como exemplo. Ele contém variáveis categóricas, como 'sex' e 'embarked',
que precisam ser codificadas para serem usadas em algoritmos de machine learning.
"""
# Carregar o dataset Titanic
import seaborn as sns
df = sns.load_dataset('titanic')

# Exibir as primeiras linhas do dataset para análise inicial
df.head()

# Filtrando colunas relevantes e removendo valores nulos
df = df[['sex', 'embarked', 'age', 'fare', 'survived']].dropna()

# Exibir as variáveis categóricas
df[['sex', 'embarked']].head()



,sex,embarked
0,male,S
1,female,C
2,female,S
3,female,S
4,male,S


In [ ]:
# Seção 4: Dividindo o Dataset em Treinamento e Teste
"""
Vamos dividir o dataset em conjuntos de treino e teste antes de aplicar qualquer transformação.
Isso garante que o modelo será avaliado em dados que ele não viu durante o treinamento.
"""
# Dividindo em variáveis independentes (X) e dependente (y)
X = df[['sex', 'embarked', 'age', 'fare']]
y = df['survived']

# Dividindo em conjuntos de treino e teste (80% treino, 20% teste)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



In [ ]:
# Seção 5: Construindo o Pipeline com OneHotEncoder e Scaler
"""
Usaremos o `Pipeline` e o `ColumnTransformer` para combinar OneHotEncoder e uma técnica de normalização
(StandardScaler ou MinMaxScaler) no mesmo fluxo de trabalho.
"""

# Definir as colunas categóricas e numéricas
categorical_cols = ['sex', 'embarked']
numeric_cols = ['age', 'fare']

# Criando o pré-processador (Pipeline para cada tipo de dado)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_cols),  # Para dados numéricos
        ('cat', OneHotEncoder(drop='first'), categorical_cols)  # Para dados categóricos
    ])

# Construindo o pipeline com o pré-processador e o modelo de classificação
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
])



In [ ]:
# Seção 6: Treinando o Modelo Usando o Pipeline
"""
Agora, usamos o Pipeline para treinar o modelo de classificação. Ele automaticamente executa o
OneHotEncoding, normaliza os dados e depois ajusta o modelo de Regressão Logística.
"""

# Ajustando (fit) o pipeline no conjunto de treino
pipeline.fit(X_train, y_train)




Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['age', 'fare']),
                                                 ('cat',
                                                  OneHotEncoder(drop='first'),
                                                  ['sex', 'embarked'])])),
                ('classifier', LogisticRegression())])

In [ ]:
# Seção 7: Avaliando o Modelo no Conjunto de Teste
"""
A avaliação do modelo também é feita usando o pipeline, garantindo que as mesmas transformações
usadas no treinamento sejam aplicadas aos dados de teste.
"""
# Prevendo no conjunto de teste
y_pred = pipeline.predict(X_test)

# Avaliando o modelo
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy do modelo: {accuracy:.2f}")



Accuracy do modelo: 0.76


In [ ]:
# Seção 8: Exercício Prático
"""
Agora, os alunos podem aplicar o mesmo Pipeline a outro dataset, como o 'tips' do seaborn,
e tentar prever se um cliente é fumante ou não.
"""
# Carregando o dataset 'tips' como exercício prático
tips_df = sns.load_dataset('tips')
tips_df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


#Explicação dos Principais Pontos:

* ColumnTransformer: Usamos o ColumnTransformer para aplicar diferentes transformações (OneHotEncoder para variáveis categóricas e StandardScaler para variáveis numéricas) a diferentes colunas de maneira eficiente.

* Pipeline: O Pipeline reúne o pré-processamento (One Hot Encoding + Normalização) e o modelo de classificação (Logistic Regression) em uma única estrutura. Isso facilita o fluxo de trabalho, garantindo que as transformações aplicadas ao conjunto de treino sejam replicadas no conjunto de teste.

* Alternância entre Normalizadores: Caso você deseje alternar entre diferentes normalizadores, basta modificar a parte que define o StandardScaler para MinMaxScaler no ColumnTransformer.

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_cols),  # Usar MinMaxScaler no lugar do StandardScaler
        ('cat', OneHotEncoder(drop='first'), categorical_cols)
    ])

* **Aplicação Automática de Transformações**: O pipeline garante que todas as transformações (encoding e normalização) sejam aplicadas de forma consistente nos dados de treinamento e teste, sem a necessidade de chamar .fit_transform() separadamente para cada parte.


### 1. **`Pipeline` e `ColumnTransformer`**

- O **`Pipeline`** é uma forma de encadear várias etapas de transformação e modelagem. No caso do exemplo, o `Pipeline` tem duas etapas:
  - Um pré-processador (`preprocessor`) para transformar os dados.
  - Um classificador (`classifier`) que realiza a classificação usando os dados já transformados.
  
- **`ColumnTransformer`** permite que diferentes transformações sejam aplicadas a diferentes colunas do DataFrame. No nosso caso:
  - **Variáveis numéricas** (`age` e `fare`) são escalonadas com `StandardScaler`.
  - **Variáveis categóricas** (`sex` e `embarked`) são codificadas com `OneHotEncoder`.

### 2. **Ajuste Automático ao DataFrame**

No momento em que o `Pipeline` é ajustado aos dados com o comando `pipeline.fit(X_train, y_train)`, o seguinte ocorre automaticamente:

- O `Pipeline` passa as colunas de **`X_train`** para o **`ColumnTransformer`**, que aplica as transformações especificadas em **`categorical_cols`** e **`numeric_cols`**.
  - **`ColumnTransformer`** separa as colunas com base nas definições feitas (quais são categóricas e quais são numéricas) e aplica as transformações adequadas.

- Uma vez que essas transformações são realizadas, o `Pipeline` continua para o próximo estágio, onde o modelo de classificação (no caso, **`LogisticRegression`**) é ajustado usando os dados transformados.

### 3. **Como o `Pipeline` entende o DataFrame**

O `Pipeline` entende o DataFrame porque:
- **`X_train`** é passado para ele durante o ajuste (`fit()`), e o `ColumnTransformer` mapeia as colunas corretamente com base nos nomes fornecidos para as colunas categóricas e numéricas.

- O **`ColumnTransformer`** cuida de aplicar as transformações somente nas colunas relevantes. As transformações acontecem com base na especificação que associou os tipos de colunas (categóricas e numéricas) aos respectivos transformadores (OneHotEncoder e StandardScaler).

### 4. **Resumo do Funcionamento**

1. **Definição de colunas**: Você especifica quais colunas serão categóricas e numéricas.
2. **Transformação**: O `ColumnTransformer` transforma cada coluna conforme o tipo definido.
3. **Encadeamento**: O `Pipeline` processa os dados em etapas sequenciais, passando as transformações para o modelo.
4. **Compatibilidade**: O `Pipeline` sabe como processar os dados do DataFrame com base nas colunas fornecidas.

Esse design é especialmente útil para garantir que as transformações corretas sejam aplicadas ao conjunto de teste (usando `.transform()` no Pipeline), evitando inconsistências entre o treinamento e teste.
